In [3]:
import ipywidgets as widgets
import redis
import json

In [4]:
import subprocess

positionv=[]
def addposition(x,y):
        positionv.append((x,y))
    
def isDetect(line):
    return line.startswith("detection=")
def toPosition(line):
      plotline=line[10:]
      plotv=plotline.split(",")
      return plotv
def apriltag(imagename):
#    imagename='/Users/aadebuger/GEXT/cloud2019/machinevision/data/tz_belt_output20190417_5/40.jpg'
    
#    lines=subprocess.getoutput('/Users/aadebuger/GEXT/github2019/apriltags/xcodebuild/Debug/apriltag_demo {0}'.format(imagename))
    lines=subprocess.getoutput('/home/aadebugergf/Ai/cloud2019/apriltags/build/apriltag_demo {0}'.format(imagename))
    linev=lines.split("\n")
    detectionv=filter(lambda item:isDetect(item),linev)
    positionv=map(lambda item:toPosition(item),detectionv)
    return list(positionv)
import numpy as np 
def getBox(plotv):
            arr2=np.asarray(plotv[1:],float)
            rect=(arr2[6],arr2[7],arr2[4]-arr2[0],arr2[1]-arr2[5])
            return (rect[0]+rect[2]/2,rect[1]+rect[3]/2)

def computeposition(x,y):
    yuandian=(853.5124299065,706.912679109)
    kx = -0.8937383178 
    ky = -0.8729680915
    scarax = yuandian[0] + kx*x 
    scaray = yuandian[1] + ky*y
    return (scarax,scaray)
r = redis.Redis(host='localhost', port=6379, db=0)
import base64
#import cStringIO
import sys
import tempfile
import time
MODEL_BASE = '/Users/aadebuger/GEXT/github2019/models/research'
MODEL_BASE = '/home/aadebugergf/Ai/github2019/models/research'


sys.path.append(MODEL_BASE)
sys.path.append(MODEL_BASE + '/object_detection')
sys.path.append(MODEL_BASE + '/slim')


import numpy as np
from PIL import Image
from PIL import ImageDraw
import tensorflow as tf
from utils import label_map_util




PATH_TO_CKPT = '/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = MODEL_BASE + '/object_detection/data/mscoco_label_map.pbtxt'

content_types = {'jpg': 'image/jpeg',
                 'jpeg': 'image/jpeg',
                 'png': 'image/png'}
extensions = sorted(content_types.keys())


def is_image():
  def _is_image(form, field):
    if not field.data:
      raise ValidationError()
    elif field.data.filename.split('.')[-1].lower() not in extensions:
      raise ValidationError()

  return _is_image





class ObjectDetector(object):

  def __init__(self):
    self.detection_graph = self._build_graph()
    self.sess = tf.Session(graph=self.detection_graph)

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    self.category_index = label_map_util.create_category_index(categories)

  def _build_graph(self):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    return detection_graph

  def _load_image_into_numpy_array(self, image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

  def detect(self, image):
    image_np = self._load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = self.detection_graph
    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    boxes = graph.get_tensor_by_name('detection_boxes:0')
    scores = graph.get_tensor_by_name('detection_scores:0')
    classes = graph.get_tensor_by_name('detection_classes:0')
    num_detections = graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = self.sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    boxes, scores, classes, num_detections = map(
        np.squeeze, [boxes, scores, classes, num_detections])

    return boxes, scores, classes.astype(int), num_detections
 

def draw_bounding_box_on_image(image, box, color='red', thickness=4):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  ymin, xmin, ymax, xmax = box
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  if (right-left)<100:
        print("too lower")
        return None
  print("rect,width="+str(right-left)+"height="+str(bottom-top))
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  print('(left, right, top, bottom)',(left, right, top, bottom))
  centerx=left+(right-left)/2
  centery=top+ (bottom-top)/2
  print("center=",centerx,"y=",centery)
  scaraxy=computeposition(centerx*4/3,centery*4/3)
  print("scara xy",scaraxy)
  return scaraxy
    
from io import BytesIO
def encode_image(image):
  image_buffer = BytesIO()
  image.save(image_buffer, format='PNG')
  imagevalue = image_buffer.getvalue()
  base64str=base64.b64encode(imagevalue).decode()

  imgstr = 'data:image/png;base64,{:s}'.format(
     base64str )
  return imgstr


def detect_objects(image_path):
  start = time.time()
  
  image = Image.open(image_path).convert('RGB')
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
  print(num_detections)
  new_images = {}
  for i in range(int(num_detections)):
    
    if scores[i] < 0.8: continue
    print("scres[i",scores[i])
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)

  result = {}
#  result['original'] = encode_image(image.copy())
#  print(new_images)
  count = 0 
  for cls, new_image in new_images.items():
    category = client.category_index[cls]['name']
    print(category)
    result[category] = encode_image(new_image)
    new_image.save("{0}.jpg".format(count))
    count=count+1
  end = time.time()
  print("execute2 time",(end-start)) 
  return result

def detect_objectsbyframe(frame):
  start = time.time()
  
#  image = Image.open(image_path).convert('RGB')
  height, width, channels = frame.shape
  image= Image.frombytes("RGB", (width,height), frame.tostring())
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
#  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
#  print(num_detections)
  new_images = {}
  
  for i in range(int(num_detections)):

    if scores[i] < 0.8: continue
    print("scres[i",scores[i])
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    box=draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)
    if box is not None:
        print("scarxbox",box)
        return (box,new_images[cls])
    return (None,None)
#  result['original'] = encode_image(image.copy())
#  print(new_images)
  return (None,None)


PATH_TO_CKPT='/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'
PATH_TO_CKPT='/home/aadebugergf/Ai/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'


PATH_TO_CKPT='/home/aadebugergf/Ai/models/fastrcnnstbmodelrtx220190703/frozen_inference_graph.pb'
client = ObjectDetector()

In [5]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052800.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        start1 = time.time()
        boxframe=frame[0:480,0:200]
        box,newframe=detect_objectsbyframe(boxframe)
        end1 = time.time()
        print("execute1 time",(end1-start1)) 
        if box is not None:
            print("box",box)

            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

execute1 time 4.593493223190308
execute1 time 0.15587568283081055
execute1 time 0.15555071830749512
execute1 time 0.1560506820678711
execute1 time 0.1261591911315918
execute1 time 0.15986061096191406
execute1 time 0.12485432624816895
execute1 time 0.16072773933410645
scres[i 0.99834836
boxes [0.69164824 0.12334522 0.98389757 0.7559917 ]
rect,width=126.52929574251175height=140.2796745300293
(left, right, top, bottom) (24.669043719768524, 151.19833946228027, 331.9911575317383, 472.2708320617676)
center= 87.9336915910244 y= 402.13099479675293
scara xy (748.7261504391585, 238.84930969470958)
scarxbox (748.7261504391585, 238.84930969470958)
execute1 time 0.1510002613067627
box (748.7261504391585, 238.84930969470958)
scres[i 0.99728
boxes [0.68568444 0.12129604 0.9836861  0.77789074]
rect,width=131.31894022226334height=143.04079055786133
(left, right, top, bottom) (24.259208142757416, 155.57814836502075, 329.12853240966797, 472.1693229675293)
center= 89.91867825388908 y= 400.64892768859863
s

scres[i 0.9041179
boxes [0.83991396 0.3912281  0.9796617  0.81032705]
too lower
execute1 time 0.16434693336486816
execute1 time 0.15892601013183594
scres[i 0.99888796
boxes [0.8404431  0.42736316 0.98321265 0.8651559 ]
too lower
execute1 time 0.158829927444458
execute1 time 0.1611649990081787
scres[i 0.9486534
boxes [0.73314667 0.4636299  0.9700655  0.9455566 ]
too lower
execute1 time 0.15655255317687988
scres[i 0.9999708
boxes [0.7123619 0.4584169 0.9835978 0.9168043]
too lower
execute1 time 0.15770649909973145
scres[i 0.99945325
boxes [0.7106454  0.47780967 0.9767483  0.92962956]
too lower
execute1 time 0.15962719917297363
scres[i 0.9999635
boxes [0.7132708  0.48149619 0.9672297  0.9193405 ]
too lower
execute1 time 0.1620790958404541
scres[i 0.99995184
boxes [0.71566427 0.5046076  0.98004985 0.9432723 ]
too lower
execute1 time 0.14392805099487305
scres[i 0.9999105
boxes [0.7188016  0.52710986 0.9757401  0.94279987]
too lower
execute1 time 0.16467857360839844
scres[i 0.99993825
boxes 

execute1 time 0.1577770709991455
execute1 time 0.15778255462646484
execute1 time 0.15990757942199707
execute1 time 0.15680718421936035
execute1 time 0.12434530258178711
execute1 time 0.15926671028137207
execute1 time 0.12545537948608398
execute1 time 0.15459561347961426
execute1 time 0.16080236434936523
execute1 time 0.158402681350708
execute1 time 0.1605057716369629
execute1 time 0.1256730556488037
execute1 time 0.15906929969787598
execute1 time 0.160491943359375
execute1 time 0.16003179550170898
execute1 time 0.1603865623474121
execute1 time 0.16177582740783691
execute1 time 0.15908145904541016
execute1 time 0.15961837768554688
execute1 time 0.1612262725830078
execute1 time 0.16103696823120117
execute1 time 0.15830373764038086
execute1 time 0.15553045272827148
execute1 time 0.15574121475219727
execute1 time 0.15857243537902832
execute1 time 0.15853548049926758
execute1 time 0.15815067291259766
execute1 time 0.15918588638305664
execute1 time 0.15868234634399414
execute1 time 0.1558151

execute1 time 0.15410375595092773
execute1 time 0.16094493865966797
execute1 time 0.15830278396606445
execute1 time 0.1621084213256836
execute1 time 0.15993690490722656
execute1 time 0.15781545639038086
execute1 time 0.15700817108154297
execute1 time 0.1577010154724121
execute1 time 0.1589970588684082
execute1 time 0.1608443260192871
execute1 time 0.15828299522399902
execute1 time 0.15802526473999023
execute1 time 0.15798187255859375
execute1 time 0.15949630737304688
execute1 time 0.15536022186279297
execute1 time 0.15445923805236816
execute1 time 0.12717437744140625
execute1 time 0.1607980728149414
execute1 time 0.1581099033355713
execute1 time 0.15897774696350098
execute1 time 0.15900206565856934
execute1 time 0.16009306907653809
capture end


In [6]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019053102.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
boxv=[]
def videooutput(boxv):
    global ivalue

    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            boxv.append(box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>120):
            break
videooutput(boxv)
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.99999857
boxes [0.6750248  0.01255637 0.9481342  0.16447271]
too lower
scres[i 0.9999982
boxes [0.66937554 0.01232546 0.94710624 0.16705246]
too lower
scres[i 0.99999654
boxes [0.6773214  0.00901999 0.9372457  0.16879973]
too lower
scres[i 0.99999845
boxes [0.6752522  0.00839564 0.9404703  0.16660728]
too lower
scres[i 0.9999981
boxes [0.6742555  0.00835825 0.940789   0.16995764]
too lower
scres[i 0.9999964
boxes [0.6701064  0.00843591 0.94430685 0.16820976]
too lower
scres[i 0.9999988
boxes [0.6735001  0.00687024 0.94488144 0.16743763]
too lower
scres[i 0.9999987
boxes [0.6732471  0.00735406 0.9466836  0.16559283]
too lower
scres[i 0.99999774
boxes [0.67278975 0.0086774  0.9432954  0.16724211]
too lower
scres[i 0.99999726
boxes [0.677674   0.00788794 0.94613445 0.16894937]
too lower
scres[i 0.9999975
boxes [0.67369837 0.00902972 0.9416625  0.16823088]
too lower
scres[i 0.99999714
boxes [0.6757288  0.00841753 0.9462322  0.16811673]
too lower
scres[i 0.99999833
boxes [0.672846

scres[i 0.99999857
boxes [0.6728114  0.00837759 0.94144404 0.16860227]
too lower
scres[i 0.99999917
boxes [0.6740134  0.00861237 0.94070214 0.16875382]
too lower
scres[i 0.99999905
boxes [0.67453647 0.0109075  0.9414382  0.17032984]
too lower
scres[i 0.9999982
boxes [0.6762573  0.00799427 0.94183666 0.16766223]
too lower
scres[i 0.99999774
boxes [0.67464197 0.00900536 0.9419629  0.16767515]
too lower
scres[i 0.99999857
boxes [0.6747206  0.00815013 0.9411529  0.16722457]
too lower
scres[i 0.99999654
boxes [0.6694953  0.01237755 0.94471246 0.16576102]
too lower
scres[i 0.999998
boxes [0.6746961  0.00975976 0.9405424  0.16814142]
too lower
scres[i 0.99999475
boxes [0.6729288  0.01327228 0.9474039  0.16823754]
too lower
scres[i 0.9999968
boxes [0.6738357  0.01011453 0.9415336  0.16945833]
too lower
scres[i 0.99999785
boxes [0.6750356  0.01033103 0.94041026 0.16880977]
too lower
scres[i 0.99999726
boxes [0.6745057  0.00658377 0.93700695 0.16480917]
too lower
scres[i 0.999998
boxes [0.673251

scres[i 0.999998
boxes [0.676031   0.00956228 0.9417868  0.16890259]
too lower
scres[i 0.9999968
boxes [0.6761278  0.00647339 0.93724966 0.1665952 ]
too lower
scres[i 0.99999833
boxes [0.67427737 0.00877207 0.9399097  0.16945897]
too lower
scres[i 0.9999981
boxes [0.6759822  0.00969602 0.9401023  0.17048687]
too lower
scres[i 0.99999714
boxes [0.67551637 0.00746044 0.93798304 0.16604017]
too lower
scres[i 0.99999857
boxes [0.6738505  0.0097665  0.9408181  0.16986439]
too lower
scres[i 0.9999964
boxes [0.67025036 0.0120769  0.94609416 0.16694495]
too lower
scres[i 0.9999989
boxes [0.6764869  0.0069589  0.94700277 0.1686674 ]
too lower
scres[i 0.99999785
boxes [0.6728424  0.00964799 0.93890625 0.16906185]
too lower
scres[i 0.99999845
boxes [0.6715845  0.00924716 0.9386022  0.16918156]
too lower
scres[i 0.9999932
boxes [0.6749331  0.0075989  0.93710685 0.1656268 ]
too lower
scres[i 0.9999982
boxes [0.6748833  0.00762407 0.94481283 0.16721258]
too lower
scres[i 0.99999726
boxes [0.6737786 